In [7]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import certifi
import pprint 
import urllib3
import html5lib
import math
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
from IPython.display import HTML
import matplotlib.pyplot as plt
import numpy as np
from itertools import product

In [8]:
URL = "https://www.ttilgb.com/uiAgs03Action/openScreen.do"
res = requests.get(URL, 'lxml', verify=False)
soup = BeautifulSoup(res.content, 'lxml')

In [9]:
def table_to_2d(table_tag):
    rowspans = []  # track pending rowspans
    rows = table_tag.find_all('tr')

    # first scan, see how many columns we need
    colcount = 0
    for r, row in enumerate(rows):
        cells = row.find_all(['td', 'th'], recursive=False)
        # count columns (including spanned).
        # add active rowspans from preceding rows
        # we *ignore* the colspan value on the last cell, to prevent
        # creating 'phantom' columns with no actual cells, only extended
        # colspans. This is achieved by hardcoding the last cell width as 1. 
        # a colspan of 0 means “fill until the end” but can really only apply
        # to the last cell; ignore it elsewhere. 
        colcount = max(
            colcount,
            sum(int(c.get('colspan', 1)) or 1 for c in cells[:-1]) + len(cells[-1:]) + len(rowspans))
        # update rowspan bookkeeping; 0 is a span to the bottom. 
        rowspans += [int(c.get('rowspan', 1)) or len(rows) - r for c in cells]
        rowspans = [s - 1 for s in rowspans if s > 1]

    # it doesn't matter if there are still rowspan numbers 'active'; no extra
    # rows to show in the table means the larger than 1 rowspan numbers in the
    # last table row are ignored.

    # build an empty matrix for all possible cells
    table = [[None] * colcount for row in rows]

    # fill matrix from row data
    rowspans = {}  # track pending rowspans, column number mapping to count
    for row, row_elem in enumerate(rows):
        span_offset = 0  # how many columns are skipped due to row and colspans 
        for col, cell in enumerate(row_elem.find_all(['td', 'th'], recursive=False)):
            # adjust for preceding row and colspans
            col += span_offset
            while rowspans.get(col, 0):
                span_offset += 1
                col += 1

            # fill table data
            rowspan = rowspans[col] = int(cell.get('rowspan', 1)) or len(rows) - row
            colspan = int(cell.get('colspan', 1)) or colcount - col
            # next column is offset by the colspan
            span_offset += colspan - 1
            value = cell.get_text()
            for drow, dcol in product(range(rowspan), range(colspan)):
                try:
                    table[row + drow][col + dcol] = value
                    rowspans[col + dcol] = rowspan
                except IndexError:
                    # rowspan or colspan outside the confines of the table
                    pass

        # update rowspan bookkeeping
        rowspans = {c: s - 1 for c, s in rowspans.items() if s > 1}

    return table

In [12]:
table1 = soup.find("td", { "class" : "h23" })

In [13]:
 pprint(table_to_2d(table1.table), width=30)

TypeError: 'module' object is not callable

In [ ]:
df = pd.DataFrame(data=res)

In [ ]:
#Or children = table.findChildren()
#for child in children:
    #print (child)

In [ ]:
#table1 = soup.find_all('tr')[0] 

In [ ]:
#df1 = pd.read_html(str(table1))
#print (df1)

In [ ]:
#table2 = soup.find_all("td", { "class" : "h23" })

In [ ]:
#df2 = pd.read_html(str(table2))

In [ ]:
#df2

In [ ]:
#soupparser = BeautifulSoup(res.content, 'html.parser')
#print(soupparser.prettify()) 

In [ ]:
table = soup.find('tbody')

In [ ]:
res = []
row = []

for tr in table.find_all('tr'):
     for td in tr.find_all('td'):
         row.append(td.text.strip())
     res.append(row)
     row = []

In [ ]:
df = pd.DataFrame(data=res)

In [ ]:
#df

In [ ]:
#df.to_excel(r"C:\Users\CodyS\Desktop\ETL-Project\data.xlsx", index = False)

In [ ]:
soupparser = BeautifulSoup(table.content, 'html.parser')
print(soupparser.prettify()) 